In [51]:
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import time
import re
import traceback
from bs4 import BeautifulSoup
from datetime import datetime
# from fake_useragent import UserAgent


In [52]:
def get_link(dir, start_date, end_date=False, Moscow_arpts=True, airport = '',):
    '''
    Функция возвращает список сслылок с сайта http://mvsm.ru/Tablo.aspx для последующего парсинга, принимая на вход 
    следующие аргументы:
    - 'dir' - направление рейса. 0 - вылет, 1 - прилет
    - start_date - начальная дата формирования ссылок (формат - YYYY-mm-dd), самое раннее значение - 2021-12-31.
    - end_date - дата завершения формирования ссылок (формат - YYYY-mm-dd). По умолчанию - текущие дата и время.
    - Moscow_arpts - по умолчанию - True, собирает информацию по аэропортам Домодедово, Шереметьево, Внуково. 
      При параметре False требуется указание параметра airports
    - airport - принимает на вход одно значение: 'DME'(Домодедово), 'SVO'(Шереметьево) или 'VKO'(Внуково).
    '''
    
    URL = 'http://mvsm.ru/Tablo.aspx'
    links_list = list()
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if end_date:
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    else:
        end_date = datetime.now()
        
    while start_date <= end_date:
        link = str(URL + (f'?Dir={dir}&Date={start_date.strftime("%Y-%m-%d")}&Hour={start_date.hour}'))
        if Moscow_arpts:
            link += (f'&City=Москва')
        if airport:
            link += (f'&Airport={airport}')
        links_list.append(link)
        
        start_date += timedelta(hours=6)
    
    return links_list 

# пример вывода URL-адреса с параметрами: http://mvsm.ru/Tablo.aspx?Dir=1&Date=2023-02-08&Hour=23&City=Москва&Airport=DME
    

In [53]:
res_links = get_link(0, '2023-02-28')

In [37]:
def get_flight_info_res(links_list, hide_joined_flights=True):
    
    check_time = None
    check_dep_airport = None
    check_arr_airport = None

    res_flights_info = pd.DataFrame()
    try:
        for link in links_list:
            try:
                response = requests.get(link)
                time.sleep(0.3)
                soup = BeautifulSoup(response.text, 'lxml')
                flights_info = soup.find_all('tr')           

                for el in list(flights_info)[30:]:
                    if el.find('a'):

                        fl_id = el.find('a', alt="Подписка").get('href')
                        fl_id = str(fl_id)[-8:]
                        if fl_id not in res_flights_info.values:                            

                            company_name = el.find('td', align='center')['title']
                            
                            company_code_pattern = str(re.search(r'\"Aircompany.aspx\?Code=\w+"', str(el)).group()).replace('"', '')
                            company_code = el.find('a', href=company_code_pattern).text
       
                            flight_number_pattern = str(re.search(r'\"FlightOnDate.aspx\?.*"', str(el)).group()).replace('"', '')
                            fl_string = str(flight_number_pattern[-26:-19])
                            flight_number = re.search(r'\s[0-9]{2,5}', fl_string).group().strip()

                            completed_flight = True
                            times_list = list()
                            time_pattern = '\d{2}.\d{2}.\d{4}\s\d{1,2}:\d{2}:\d{2}'
                            for i in el.find_all('td', align="center"):
                                if re.findall(time_pattern, i.text):
                                    times_list.append(str(re.search(time_pattern, i.text).group()))
                                if i.get_text(strip=True) == 'Отменен':
                                    completed_flight = False
                                    break
                                pass
                            plan_time, fact_time = times_list

                            aircraft = None
                            aircraft_pattern = '[АБВЕИЛМСABCDEFJLM]{1}\d\w{1,2}|32[01]{1}|Boeing\s\w+|Sukhoi\s\w+\s\w+|SSJ\d{2,3}'
                            for j in el.find_all('td', align="center"):
                                if re.search(aircraft_pattern, j.text):
                                    aircraft = (re.search(aircraft_pattern, j.text).group().replace('Б', 'B')
                                                .replace('В', 'B').replace('Е', 'E').replace('oeing', '').replace('320', 'A 320')
                                                .replace('AA', 'A').replace('А', 'A').replace('irbus', '')
                                                .replace('M83', 'MD83').replace('Sukhoi Superjet', 'SSJ')
                                                .replace(' ', '').replace('21N', '321').replace('20N', '320')
                                                .replace('B77W', 'B777').replace('J200', 'CRJ200').replace('E75L', 'E175')
                                                .replace('b38m', 'B738').replace('L76', 'Il76').replace('B39M', 'B739')
                                               )
                                    break
                                pass                                    
                                                      
                            airports_list = list()
                            for i in el.find_all('td', align="left"):
                                airports_list.append(str(i.get_text(strip=True)))       
                                
                            dep_airport = airports_list[0].upper()
                            arr_airport = airports_list[2].upper()
                    
                            if hide_joined_flights:
                                if (fact_time != check_time) and (arr_airport != check_arr_airport) and (dep_airport != check_dep_airport):            
                                    row = {'Flight_id': fl_id, 'Company_name': company_name, 'Company_code': company_code,
                                           'Flight_number': flight_number, 'Plan_time': plan_time, 'Fact_time': fact_time,
                                           'Completed_flight': completed_flight, 'Aircraft': aircraft, 
                                           'Departure_airport': dep_airport, 'Arrival_airport': arr_airport}

                                    check_time = fact_time
                                    check_arr_airport = arr_airport
                                    check_dep_airport = dep_airport
                                    
                                    res_flights_info = pd.concat([res_flights_info, pd.DataFrame([row])])
                                    continue

                            else:
                                row = {'Flight_id': fl_id, 'Company_name': company_name, 'Company_code': company_code, 'Flight_number': flight_number,
                                       'Plan_time': plan_time, 'Fact_time': fact_time, 'Completed_flight': completed_flight, 'Aircraft': aircraft,
                                       'Departure_airport': dep_airport, 'Arrival_airport': arr_airport}
                                
                                res_flights_info = pd.concat([res_flights_info, pd.DataFrame([row])])
                                pass

            except Exception as e:
                print(link)
                pass
                            
    finally:
        return res_flights_info

In [38]:
result = get_flight_info_res(res_links, False)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_18032\1867251120.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if fl_id not in res_flights_info.values:


http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-20&Hour=0&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-20&Hour=6&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-20&Hour=12&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-20&Hour=18&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-21&Hour=0&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-21&Hour=6&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-21&Hour=12&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-21&Hour=18&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-22&Hour=0&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-22&Hour=6&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-22&Hour=12&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-22&Hour=18&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-23&Hour=0&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-23&Hour=6&City=Москва
http://mvsm.ru/Tablo.aspx?Dir=0&Date=2023-02-23&Hour=12&

In [88]:
result = pd.DataFrame(result)

In [22]:
result.sort_values(['Plan_time', 'Fact_time', 'Departure_airport', 'Arrival_airport', 'Aircraft']).tail(50)

,Flight_id,Company_name,Company_code,Flight_number,Plan_time,Fact_time,Completed_flight,Aircraft,Departure_airport,Arrival_airport
0,19374327,"Indonesia AirAsia Extra, PT",XT,2141,28.02.2023 8:40:00,28.02.2023 9:00:00,True,None,МОСКВА (ДОМОДЕДОВО),ВОЛГОГРАД (ВОЛГОГРАД)
0,19374328,S7 Airlines,S7,2141,28.02.2023 8:40:00,28.02.2023 9:00:00,True,None,МОСКВА (ДОМОДЕДОВО),ВОЛГОГРАД (ВОЛГОГРАД)
0,19365551,Северный Ветер (Nordwind Airlines),N4,1401,28.02.2023 8:40:00,28.02.2023 9:05:36,True,B737,МОСКВА (ШЕРЕМЕТЬЕВО),БОХТАР (БОХТАР)
0,19365555,Аэрофлот,SU,1758,28.02.2023 8:45:00,28.02.2023 8:50:33,True,A320,МОСКВА (ШЕРЕМЕТЬЕВО),ВОЛГОГРАД (ВОЛГОГРАД)
0,19374329,Gulf Air,GF,013,28.02.2023 8:45:00,28.02.2023 9:36:00,True,None,МОСКВА (ДОМОДЕДОВО),МАНАМА (БАХРЕЙН)
0,19365556,Победа,DP,6923,28.02.2023 8:45:00,28.02.2023 9:57:06,True,B737,МОСКВА (ШЕРЕМЕТЬЕВО),МАХАЧКАЛА (УЙТАШ)
0,19365559,Аэрофлот,SU,6405,28.02.2023 8:50:00,28.02.2023 8:50:00,False,SSJ100,МОСКВА (ШЕРЕМЕТЬЕВО),СЫКТЫВКАР (СЫКТЫВКАР)
0,19365561,ОАО 'Авиакомпания 'Россия',FV,6405,28.02.2023 8:50:00,28.02.2023 8:50:00,True,SSJ100,МОСКВА (ШЕРЕМЕТЬЕВО),СЫКТЫВКАР (СЫКТЫВКАР)
0,19377589,Аэрофлот,SU,1780,28.02.2023 8:50:00,28.02.2023 8:55:44,True,A320,МОСКВА (ШЕРЕМЕТЬЕВО),СЫКТЫВКАР (СЫКТЫВКАР)
0,19374331,Ижавиа,I8,302,28.02.2023 8:50:00,28.02.2023 9:05:00,True,None,МОСКВА (ДОМОДЕДОВО),ИЖЕВСК (ИЖЕВСК)


In [39]:
result.shape

(5102, 10)

In [40]:
result.to_csv('MOW_DEPARTURES_20230220_20230228.csv', sep='\t', encoding='utf-8')

In [41]:
result.to_excel('MOW_DEPARTURES_20230220_20230228.xlsx', encoding='utf-8')